# 🎬 ГОДЖО 30 СЕК — ПРОСТАЯ ВЕРСИЯ БЕЗ COMFYUI

**Что делает этот notebook:**
- Генерирует анимацию с AnimateDiff или статичные кадры
- Применяет upscale через Real-ESRGAN (опционально)
- Интерполирует кадры через RIFE для плавности
- Создает финальное видео

**Инструкция:**
1. Загрузите датасет с моделями в Kaggle
2. Настройте параметры в ячейке ниже
3. Запустите все ячейки по порядку (Run All)

In [ ]:
# === РАННЯЯ НАСТРОЙКА ОКРУЖЕНИЯ / ДИАГНОСТИКА ===
# Устанавливаем важные переменные окружения ДО любых heavy-импортов,
# чтобы избежать сообщений о дублирующей регистрации CUDA-плагинов (cuFFT/cuDNN/cuBLAS) от XLA/JAX/TensorFlow.
import os
import importlib
_changed_env = False

def _set_env_if_needed(k, v):
    global _changed_env
    if os.environ.get(k) != v:
        os.environ[k] = v
        _changed_env = True

# Подавляем TensorFlow info/warnings, отключаем предалокацию XLA клиента
_set_env_if_needed('TF_CPP_MIN_LOG_LEVEL', '3')
_set_env_if_needed('XLA_PYTHON_CLIENT_PREALLOCATE', 'false')
_set_env_if_needed('XLA_PYTHON_CLIENT_MEM_FRACTION', '0.0')
# Принудительно переключаем JAX на CPU, если он установлен — чтобы он не регистрировал GPU-плагины
_set_env_if_needed('JAX_PLATFORM_NAME', 'cpu')
_set_env_if_needed('JAX_PLATFORMS', 'cpu')

# Попытаемся мягко подавить absl-логи (если absl присутствует)
try:
    import absl.logging
    absl.logging._warn_preinit_stderr = False
    absl.logging.set_verbosity(absl.logging.WARNING)
except Exception:
    pass

# Диагностика: есть ли установленные библиотеки, которые могут вызвать XLA/JAX/TensorFlow логирование
_found = {}
for mod in ('jax', 'jaxlib', 'tensorflow', 'torch_xla'):
    _found[mod] = importlib.util.find_spec(mod) is not None

print('ENV diagnostic:')
print('  TF_CPP_MIN_LOG_LEVEL=', os.environ.get('TF_CPP_MIN_LOG_LEVEL'))
print('  XLA_PYTHON_CLIENT_PREALLOCATE=', os.environ.get('XLA_PYTHON_CLIENT_PREALLOCATE'))
print('  XLA_PYTHON_CLIENT_MEM_FRACTION=', os.environ.get('XLA_PYTHON_CLIENT_MEM_FRACTION'))
print('  JAX_PLATFORM_NAME=', os.environ.get('JAX_PLATFORM_NAME'))
print('  JAX_PLATFORMS=', os.environ.get('JAX_PLATFORMS'))
print('Detected potentially conflicting packages:')
for k, v in _found.items():
    print(f'  {k}:', 'present' if v else 'not found')

if _changed_env:
    print('\n⚠️ Важно: переменные окружения изменены — перезапустите kernel (Restart Kernel) перед дальнейшим импортом heavy-библиотек, чтобы изменения вступили в силу и сообщения XLA/TensorFlow не появлялись.')
else:
    print('\nℹ️ Переменные окружения уже установлены.')


In [ ]:
# === KAGGLE SETUP & CHECKS ===
# Эта ячейка автоматически выполняет базовую проверку для запуска в Kaggle и даёт инструкции.
import os
import sys
import textwrap

# Защитная проверка: определяем print_separator если ячейка запускается отдельно
if 'print_separator' not in globals():
    def print_separator(nl_before=True):
        SEP = '=' * 60
        if nl_before:
            print('\n' + SEP)
        else:
            print(SEP + '\n')

is_kaggle = any(p.startswith('/kaggle') for p in (os.getcwd(),)) or os.environ.get('KAGGLE_KERNEL_RUN_TYPE') is not None
print_separator()
print('🧭 KAGGLE CHECK')
print_separator(nl_before=False)
print('Running in Kaggle environment:', is_kaggle)

# GPU check (non-blocking): try to detect torch without blocking the notebook
import importlib.util
import subprocess
import json

try:
    if importlib.util.find_spec('torch') is None:
        print('Torch not installed — GPU check skipped (will be checked again in import cell).')
    else:
        print('Torch package detected — running lightweight subprocess check (timeout 12s) to avoid hanging import...')
        try:
            cmd = [sys.executable, '-c', 'import torch, json; print(json.dumps({"cuda": torch.cuda.is_available(), "device_count": torch.cuda.device_count(), "device_name": (torch.cuda.get_device_name(0).strip() if torch.cuda.is_available() else "no gpu")}))']
            # timeout can be adjusted via env var KAGGLE_TORCH_CHECK_TIMEOUT (seconds)
            _timeout = int(os.environ.get('KAGGLE_TORCH_CHECK_TIMEOUT', '12'))
            print(f'(torch subprocess timeout set to {_timeout}s)')
            res = subprocess.run(cmd, capture_output=True, text=True, timeout=_timeout)
            if res.returncode == 0 and res.stdout:
                try:
                    info = json.loads(res.stdout.strip())
                    cuda_avail = info.get('cuda', False)
                    device_name = info.get('device_name', 'no gpu')
                    print(f'GPU available: {cuda_avail} — {device_name}')
                except Exception:
                    print('✓ torch detected but could not parse subprocess output:', res.stdout.strip())
            else:
                print('⚠️ torch subprocess failed or produced no output. stderr:', res.stderr.strip())
        except subprocess.TimeoutExpired:
            print('⚠️ Torch import timed out (subprocess). Skipping GPU check to avoid hanging the notebook.')
except Exception as _e:
    print('Torch check skipped (error):', _e)

# Check for Hugging Face token in environment or common Kaggle input path
hf_token = os.environ.get('HUGGINGFACE_HUB_TOKEN') or os.environ.get('HF_TOKEN')
# First try the canonical path /kaggle/input/hf-token/token.txt
hf_token_file = '/kaggle/input/hf-token/token.txt'
found_token_path = None
if not hf_token and os.path.exists(hf_token_file):
    try:
        with open(hf_token_file, 'r', encoding='utf-8') as f:
            hf_token = f.read().strip()
            os.environ['HUGGINGFACE_HUB_TOKEN'] = hf_token
            found_token_path = hf_token_file
            print(f'✓ Hugging Face token found in {hf_token_file} and set to HUGGINGFACE_HUB_TOKEN')
    except Exception as e:
        print('⚠️ Не удалось прочитать token file:', e)

# If still not found, search all /kaggle/input/** for token*.txt (flexible)
if not hf_token:
    try:
        import glob
        candidates = glob.glob('/kaggle/input/**/token*.txt', recursive=True)
        if candidates:
            # pick the first reasonable candidate
            for c in candidates:
                try:
                    with open(c, 'r', encoding='utf-8') as f:
                        t = f.read().strip()
                    if t:
                        hf_token = t
                        os.environ['HUGGINGFACE_HUB_TOKEN'] = hf_token
                        found_token_path = c
                        print(f'✓ Hugging Face token found in {c} and set to HUGGINGFACE_HUB_TOKEN')
                        break
                except Exception:
                    continue
    except Exception as _e:
        print('⚠️ Ошибка при поиске token files:', _e)

if hf_token:
    print('✓ Hugging Face token available via environment.')
    if found_token_path:
        print('  (token loaded from:', found_token_path + ')')
else:
    print('\n⚠️ Hugging Face token not found.')
    print(textwrap.dedent('''
    Чтобы загрузить модели с Hugging Face в Kaggle, добавьте ваш токен следующим образом:

    1) В интерфейсе Kaggle: Add Data -> создайте dataset с файлом token.txt (содержит ваш токен) и подключите его к ноутбуку как /kaggle/input/hf-token
    2) Либо установите переменную окружения прямо в ноутбуке (выполните в отдельной ячейке):

       import os
       os.environ['HUGGINGFACE_HUB_TOKEN'] = 'ВАШ_ТОКЕН'

    После установки токена перезапустите kernel.
    '''))

# ipywidgets hint
try:
    import ipywidgets  # type: ignore
    print('✓ ipywidgets доступен')
except Exception:
    print('\n⚠️ ipywidgets не установлен — GUI для промптов не будет работать.')
    print('  Установите ipywidgets в отдельной ячейке и перезапустите kernel:')
    print('\n```bash\n!pip install ipywidgets -q\n```\n')

# Quick safe defaults suggestion for Kaggle (optional)
if is_kaggle:
    print('\n💡 Рекомендации для запуска в Kaggle:')
    print('  - Включите GPU в Settings -> Accelerator -> GPU')
    print('  - Для теста используйте небольшие параметры: WIDTH=256, HEIGHT=256, NUM_FRAMES=4, STEPS=15')
    print('  - Если модель не помещается в память, уменьшите WIDTH/NUM_FRAMES или используйте model_cpu_offload() (pipeline уже настраивается с этим флагом)')

print_separator()


In [ ]:
# === SMOKE TEST (проверка наличия prompts.json и Hugging Face token) ===
import glob, os, json
print_separator()
print('🔎 SMOKE TEST — проверка prompts.json и Hugging Face token')
print_separator(nl_before=False)

# Покажем подключённые input'ы
inputs = glob.glob('/kaggle/input/*') if os.path.exists('/kaggle/input') else []
print('Connected input datasets:', inputs)

# Ищем prompts.json
prompts_candidates = glob.glob('/kaggle/input/**/prompts.json', recursive=True) if os.path.exists('/kaggle/input') else []
if not prompts_candidates and os.path.exists('/kaggle/working/prompts.json'):
    prompts_candidates = ['/kaggle/working/prompts.json']

if prompts_candidates:
    p = prompts_candidates[0]
    print('✓ prompts.json found at:', p)
    try:
        with open(p, 'r', encoding='utf-8') as f:
            data = json.load(f)
        print('  keys:', list(data.keys()))
        # preview safe snippets
        base = (data.get('BASE_PROMPT') or '')[:200]
        motion = (data.get('MOTION_PROMPT') or '')[:120]
        print('  BASE_PROMPT preview:', base)
        print('  MOTION_PROMPT preview:', motion)
    except Exception as e:
        print('⚠️ Ошибка при чтении prompts.json:', e)
else:
    print('⚠️ prompts.json not found in /kaggle/input or /kaggle/working. Attach dataset or upload file.')
    print("  Tips: Add Data -> attach 'noxfvr/comfy-gojo-dataset' or upload prompts.json to working dir.")

# Проверяем Hugging Face token (без вывода полного значения)
token = os.environ.get('HUGGINGFACE_HUB_TOKEN') or os.environ.get('HF_TOKEN')
if token:
    print('\n✓ HUGGINGFACE_HUB_TOKEN found in environment (masked):')
    try:
        print('  length:', len(token), 'prefix:', token[:6] + '...')
    except Exception:
        print('  (cannot preview token)')
    # Пытаемся валидацию, если установлен huggingface_hub
    try:
        from huggingface_hub import HfApi
        api = HfApi()
        info = api.whoami(token=token)
        user = info.get('name') or info.get('user') or info
        print('  HF token appears valid; user:', user)
    except Exception as e:
        print('  Could not validate token with huggingface_hub (not installed or error):', e)
        print("  To validate: run '!pip install -q huggingface_hub' and re-run this cell")
else:
    print('\n⚠️ HUGGINGFACE_HUB_TOKEN not found in environment.')
    print('  If you have hf-token dataset attached, ensure it contains token.txt and restart the kernel.')

print_separator()


In [ ]:
# ============================================
# 📝 НАСТРОЙКИ - МЕНЯЙ ТОЛЬКО ЭТО!
# ============================================
# Константы по умолчанию для промптов (используются в нескольких местах)
DEFAULT_BASE_PROMPT = "cinematic portrait of gojo satoru, white spiky hair, black blindfold, confident expression, anime style, highly detailed, 8k, professional lighting"
DEFAULT_NEGATIVE_PROMPT = "blurry, deformed, low quality, watermark, text, bad anatomy, multiple heads, duplicate"

PROMPT = DEFAULT_BASE_PROMPT
NEGATIVE_PROMPT = DEFAULT_NEGATIVE_PROMPT

# Режим работы
USE_ANIMATEDIFF = True  # True = настоящая анимация, False = статичные кадры + RIFE

# Параметры генерации
WIDTH = 512
HEIGHT = 768
NUM_FRAMES = 16 if USE_ANIMATEDIFF else 8  # AnimateDiff: 16-24, обычный: 8-12
STEPS = 25 if USE_ANIMATEDIFF else 20  # Для анимации нужно больше steps
CFG_SCALE = 7.5 if USE_ANIMATEDIFF else 7
FPS = 8  # FPS для финального видео

# Интерполяция RIFE (опционально)
USE_RIFE = True  # Применить RIFE для ещё более плавной анимации
RIFE_EXP = 4 if USE_ANIMATEDIFF else 5  # AnimateDiff: 4 (16→256), обычный: 5 (8→256)
# Имя файла модели для апскейла (Real-ESRGAN)
UPSCALE_MODEL_NAME = '4x-UltraSharp.pth'
# ============================================

# === PROMPTS — ВАШИ PROMPT'Ы ДЛЯ ИЗОБРАЖЕНИЯ И АНИМАЦИИ (перемещена) ===
# Эта ячейка теперь находится прямо после блока настроек. Редактируйте здесь BASE/MOTION/EXTRA/NEGATIVE
import os
from IPython.display import display, clear_output

# Путь для сохранения промптов
PROMPTS_FILE = os.path.join(os.getcwd(), 'prompts.json')

# Набор пресетов: имя -> (base, motion, extra, negative)
PRESETS = {
    'default': (
        DEFAULT_BASE_PROMPT,
        "turning head, hair flowing, smooth motion",
        "dramatic rim lighting, soft bloom, depth of field",
        DEFAULT_NEGATIVE_PROMPT
    ),
    'slow pan': (
        "cinematic portrait, highly detailed, 8k, beautiful face",
        "slow camera pan left, subtle head turn",
        "soft warm lighting, cinematic",
        "blurry, low quality, watermark, text"
    ),
    'head turn': (
        "close-up portrait, detailed, professional lighting",
        "turning head to left then right, hair movement",
        "rim light, subtle bloom",
        "multiple heads, deformed, watermark"
    ),
    'blinking': (
        "portrait, soft lighting, anime style",
        "subtle blink, small head tilt",
        "soft bokeh, cinematic lighting",
        "blurry, artifact, watermark"
    )
}

# Загружаем пресет (если PROMPT уже задан, пытаемся сопоставить)
def load_preset(name):
    if name in PRESETS:
        base, motion, extra, negative = PRESETS[name]
        return {'BASE_PROMPT': base, 'MOTION_PROMPT': motion, 'EXTRA_PROMPT': extra, 'NEGATIVE_PROMPT': negative}
    return None

# Сохраняем в prompts.json
def save_prompts_file(data, path=PROMPTS_FILE):
    try:
        import json
        with open(path, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=2)
        print(f"✓ Prompts saved to {path}")
    except Exception as e:
        print('⚠️ Не удалось сохранить prompts.json:', e)

# Загружаем из prompts.json
def load_prompts_file(path=PROMPTS_FILE):
    try:
        import json
        if os.path.exists(path):
            with open(path, 'r', encoding='utf-8') as f:
                return json.load(f)
    except Exception as e:
        print('⚠️ Не удалось загрузить prompts.json:', e)
    return None

# Попытка подключить ipywidgets; если недоступен — выводим fallback инструкции
def create_prompts_gui():
    try:
        import ipywidgets as widgets
    except Exception:
        print('⚠️ ipywidgets не установлен — GUI недоступен. Установите: pip install ipywidgets')
        # Fallback: просто создаём PROMPТ из текущ констант
        parts = [p for p in (globals().get('PROMPT', ''), '') if p]
        globals()['PROMPT'] = globals().get('PROMPT', '')
        globals()['NEGATIVE_PROMPT'] = globals().get('NEGATIVE_PROMPT', '')
        print('\nPROMPT (fallback):', globals()['PROMPT'])
        return

    # Виджеты
    preset_dropdown = widgets.Dropdown(options=list(PRESETS.keys()), value='default', description='Preset:')
    base_ta = widgets.Textarea(value=PRESETS['default'][0], description='Base:', layout=widgets.Layout(width='100%', height='80px'))
    motion_ta = widgets.Textarea(value=PRESETS['default'][1], description='Motion:', layout=widgets.Layout(width='100%', height='60px'))
    extra_ta = widgets.Textarea(value=PRESETS['default'][2], description='Extra:', layout=widgets.Layout(width='100%', height='60px'))
    negative_ta = widgets.Textarea(value=PRESETS['default'][3], description='Negative:', layout=widgets.Layout(width='100%', height='60px'))

    save_btn = widgets.Button(description='Save to prompts.json', button_style='success')
    load_btn = widgets.Button(description='Load from prompts.json')
    update_btn = widgets.Button(description='Update PROMPT', button_style='primary')
    out = widgets.Output()

    # Обработчики
    def on_preset_change(change):
        if change['type'] == 'change' and change['name'] == 'value':
            vals = load_preset(change['new'])
            if vals:
                base_ta.value = vals['BASE_PROMPT']
                motion_ta.value = vals['MOTION_PROMPT']
                extra_ta.value = vals['EXTRA_PROMPT']
                negative_ta.value = vals['NEGATIVE_PROMPT']

    def on_save_clicked(b):
        data = {
            'BASE_PROMPT': base_ta.value,
            'MOTION_PROMPT': motion_ta.value,
            'EXTRA_PROMPT': extra_ta.value,
            'NEGATIVE_PROMPT': negative_ta.value
        }
        save_prompts_file(data)

    def on_load_clicked(b):
        data = load_prompts_file()
        if data:
            base_ta.value = data.get('BASE_PROMPT', base_ta.value)
            motion_ta.value = data.get('MOTION_PROMPT', motion_ta.value)
            extra_ta.value = data.get('EXTRA_PROMPT', extra_ta.value)
            negative_ta.value = data.get('NEGATIVE_PROMPT', negative_ta.value)
            with out:
                clear_output()
                print('✓ Prompts loaded into GUI (not yet applied)')
        else:
            with out:
                clear_output()
                print('⚠️ prompts.json не найден или недоступен')

    def on_update_clicked(b):
        # Собираем итоговый PROMPT
        parts = [p.strip() for p in (base_ta.value, motion_ta.value, extra_ta.value) if p and p.strip()]
        final = ', '.join(parts)
        globals()['PROMPT'] = final
        globals()['NEGATIVE_PROMPT'] = negative_ta.value
        with out:
            clear_output()
            print('✓ PROMPT обновлён')
            print('\nPROMPT:')
            print(final)
            print('\nNEGATIVE_PROMPT:')
            print(negative_ta.value)

    preset_dropdown.observe(on_preset_change)
    save_btn.on_click(on_save_clicked)
    load_btn.on_click(on_load_clicked)
    update_btn.on_click(on_update_clicked)

    # Layout
    controls = widgets.VBox([
        preset_dropdown,
        base_ta,
        motion_ta,
        extra_ta,
        negative_ta,
        widgets.HBox([update_btn, save_btn, load_btn]),
        out
    ])

    display(controls)
    # Попробуем загрузить prompts.json в GUI при инициализации
    data = load_prompts_file()
    if data:
        base_ta.value = data.get('BASE_PROMPT', base_ta.value)
        motion_ta.value = data.get('MOTION_PROMPT', motion_ta.value)
        extra_ta.value = data.get('EXTRA_PROMPT', extra_ta.value)
        negative_ta.value = data.get('NEGATIVE_PROMPT', negative_ta.value)
        with out:
            print('✓ prompts.json загружен в GUI')

# Вызываем создание GUI
create_prompts_gui()


In [ ]:
# === ИНИЦИАЛИЗАЦИЯ ===
import os
import time
from IPython.display import FileLink, display

WORKSPACE = "/kaggle/working"
FRAMES_DIR = f"{WORKSPACE}/frames"
DATASET_DIR = "/kaggle/input/comfyui-models-gojo"

os.makedirs(FRAMES_DIR, exist_ok=True)
print("✓ Папки созданы")

In [ ]:
# === УСТАНОВКА ЗАВИСИМОСТЕЙ ===
print("🔧 Проверка и установка зависимостей...\n")

import sys
import importlib.util

def check_package(package_name):
    """Быстрая проверка наличия пакета без полного импорта"""
    return importlib.util.find_spec(package_name) is not None

# Список необходимых пакетов
packages_to_install = []

if not check_package("diffusers"):
    packages_to_install.append("diffusers[torch]")
    packages_to_install.append("transformers")
    packages_to_install.append("accelerate")
else:
    print("✓ diffusers уже установлен")

if not check_package("cv2"):
    packages_to_install.append("opencv-python")
else:
    print("✓ opencv-python уже установлен")

if USE_ANIMATEDIFF and not check_package("imageio"):
    packages_to_install.append("imageio")
    packages_to_install.append("imageio-ffmpeg")
elif USE_ANIMATEDIFF:
    print("✓ imageio уже установлен")

# Устанавливаем ipywidgets для GUI, если его нет
if not check_package("ipywidgets"):
    packages_to_install.append("ipywidgets")
else:
    print("✓ ipywidgets уже установлен")

# Устанавливаем все пакеты одной командой (через subprocess для переносимости)
if packages_to_install:
    print(f"\nУстановка: {', '.join(packages_to_install)}...")
    import subprocess
    # Используем Python interpreter для гарантированной установки в текущее окружение
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q'] + packages_to_install)
    print("✓ Установка завершена!")
    # При автоматической установке ipywidgets потребуется перезапуск kernel для корректной работы GUI
    if 'ipywidgets' in packages_to_install:
        print('\n⚠️ ipywidgets установлен — перезапустите kernel (Restart Kernel) в интерфейсе Kaggle, чтобы GUI заработал корректно.')

print("\n✓ Зависимости готовы!")

In [ ]:
#=== ПОДАВЛЕНИЕ ШУМА ОТ XLA / TensorFlow / absl ===
# Устанавливаем переменные окружения ДО импортов, чтобы избежать дублирующей регистрации CUDA-плагинов
import os
# Скрыть INFO/WARNING TensorFlow-логи (если TF подгружается косвенно)
os.environ.setdefault('TF_CPP_MIN_LOG_LEVEL', '3')
# Отключить предалокацию памяти у XLA клиента (если установлен jax/xla)
os.environ.setdefault('XLA_PYTHON_CLIENT_PREALLOCATE', 'false')
os.environ.setdefault('XLA_PYTHON_CLIENT_MEM_FRACTION', '0.0')

# Попытаемся подавить предупреждения absl уже на этапе импорта (без падений, если absl нет)
try:
    import absl.logging
    # Не писать предварительные сообщения в stderr
    absl.logging._warn_preinit_stderr = False
    absl.logging.set_verbosity(absl.logging.WARNING)
except Exception:
    pass

# === ИМПОРТ БИБЛИОТЕК ===
# Защитная проверка: если функция print_separator не была ранее определена (ячейка не выполнялась),
# определим простую совместимую версию, чтобы избежать NameError при запуске отдельной ячейки.
if "print_separator" not in globals():
    def print_separator(nl_before=True):
        SEP = '=' * 60
        if nl_before:
            print('\n' + SEP)
        else:
            print(SEP + '\n')

print_separator()
print("🎨 Загрузка библиотек (это может занять ~30 сек)...\n")

import torch
from PIL import Image

print("✓ PyTorch и PIL загружены")
print(f"✓ CUDA доступна: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✓ GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
# === ЗАГРУЗКА ANIMATEDIFF PIPELINE (отдельная ячейка) ===
# Запустите эту ячейку перед ячейкой генерации, если хотите отдельно подготовить модель и сэкономить время при повторной генерации.
if 'USE_ANIMATEDIFF' in globals() and USE_ANIMATEDIFF:
    if 'pipe' in globals() and 'adapter' in globals():
        print('✓ AnimateDiff pipeline уже загружен (pipe, adapter в globals).')
    else:
        print('Загрузка AnimateDiff pipeline...')
        try:
            from diffusers import AnimateDiffPipeline, MotionAdapter, EulerDiscreteScheduler
            from diffusers.utils import export_to_video
            import torch

            adapter = MotionAdapter.from_pretrained(
                "guoyww/animatediff-motion-adapter-v1-5-2",
                torch_dtype=torch.float16
            )

            # AnimateDiff работает с SD 1.5
            pipe = AnimateDiffPipeline.from_pretrained(
                "runwayml/stable-diffusion-v1-5",
                motion_adapter=adapter,
                torch_dtype=torch.float16
            ).to("cuda")

            pipe.scheduler = EulerDiscreteScheduler.from_config(pipe.scheduler.config)
            pipe.enable_vae_slicing()
            pipe.enable_model_cpu_offload()

            print('✓ AnimateDiff готов! (adapter & pipe загружены)')
        except Exception as _e:
            print('⚠️ Не удалось загрузить AnimateDiff pipeline:', _e)
            print('  Проверьте установку diffusers, доступ к интернету и наличие CUDA/GPU.')

In [ ]:
# === ГЕНЕРАЦИЯ КАДРОВ / АНИМАЦИИ ===
# Защитная проверка: если пользователь выполнил только эту ячейку, то установим безопасные значения по умолчанию
import os
import time
_defaults = {
    'PROMPT': "cinematic portrait of gojo satoru, white spiky hair, black blindfold, confident expression, anime style, highly detailed, 8k, professional lighting",
    'NEGATIVE_PROMPT': "blurry, deformed, low quality, watermark, text, bad anatomy, multiple heads, duplicate",
    'USE_ANIMATEDIFF': True,
    'USE_RIFE': True,
    'WIDTH': 512,
    'HEIGHT': 768,
    'NUM_FRAMES': 16,
    'STEPS': 25,
    'CFG_SCALE': 7.5,
    'FPS': 8,
    'RIFE_EXP': 4,
    'WORKSPACE': os.getcwd()
}
for _k, _v in _defaults.items():
    if _k not in globals():
        globals()[_k] = _v
        print(f"⚠️ Переменная {_k} не найдена — установлено значение по умолчанию: {_v}")

# Убедимся, что директория для кадров существует
if 'FRAMES_DIR' not in globals():
    FRAMES_DIR = f"{WORKSPACE}/frames"
os.makedirs(FRAMES_DIR, exist_ok=True)

print_separator()
print(f"🎨 {'ГЕНЕРАЦИЯ АНИМАЦИИ' if USE_ANIMATEDIFF else 'ГЕНЕРАЦИЯ КАДРОВ'}")
print_separator(nl_before=False)
print(f"Промпт: {PROMPT[:80]}...")
print(f"Размер: {WIDTH}x{HEIGHT}, Steps: {STEPS}, CFG: {CFG_SCALE}")
print(f"Кадров: {NUM_FRAMES}\n")

if USE_ANIMATEDIFF:
    # === РЕЖИМ ANIMATEDIFF - НАСТОЯЩАЯ АНИМАЦИЯ ===
    # Теперь генерация использует уже загруженный `pipe` (если он есть). Если нет — подсказываем запустить ячейку загрузки.
    if 'pipe' not in globals():
        print("⚠️ AnimateDiff pipeline не загружен. Запустите отдельную ячейку 'ЗАГРУЗКА ANИМАЦИИ PIPELINE' перед генерацией или выполните эту ячейку, чтобы загрузить его автоматически.")
        # Попробуем всё же подгрузить inline (fallback):
        try:
            from diffusers import AnimateDiffPipeline, MotionAdapter, EulerDiscreteScheduler
            from diffusers.utils import export_to_video
            import torch

            print('Загрузка AnimateDiff pipeline (fallback inline)...')
            adapter = MotionAdapter.from_pretrained(
                "guoyww/animatediff-motion-adapter-v1-5-2",
                torch_dtype=torch.float16
            )
            pipe = AnimateDiffPipeline.from_pretrained(
                "runwayml/stable-diffusion-v1-5",
                motion_adapter=adapter,
                torch_dtype=torch.float16
            ).to("cuda")
            pipe.scheduler = EulerDiscreteScheduler.from_config(pipe.scheduler.config)
            pipe.enable_vae_slicing()
            pipe.enable_model_cpu_offload()

        except Exception as e:
            raise RuntimeError('Не удалось подгрузить AnimateDiff pipeline автоматически: ' + str(e))

    print('Генерация анимации (это займет ~3-7 минут)...')

    start_time = time.time()

    # Генерируем анимацию
    output = pipe(
        prompt=PROMPT,
        negative_prompt=NEGATIVE_PROMPT,
        num_frames=NUM_FRAMES,
        width=WIDTH,
        height=HEIGHT,
        num_inference_steps=STEPS,
        guidance_scale=CFG_SCALE,
        generator=torch.Generator("cuda").manual_seed(42)
    )

    frames = output.frames[0]
    total_gen_time = time.time() - start_time

    print(f"\n✅ Анимация готова за {total_gen_time:.1f}s!\n")

    # Сохраняем кадры
    for i, frame in enumerate(frames):
        frame.save(f"{FRAMES_DIR}/{i}.png")

    # Создаем базовое видео
    base_video = f"{WORKSPACE}/ANIMATED_BASE.mp4"
    export_to_video(frames, base_video, fps=FPS)
    print(f"✓ Базовое видео сохранено ({len(frames)} кадров, {FPS} fps)")

    del pipe, adapter
    torch.cuda.empty_cache()

else:
    # === РЕЖИМ СТАТИЧНЫХ КАДРОВ ===
    from diffusers import StableDiffusionXLPipeline

    # Загружаем модель из датасета или HuggingFace
    model_path = f"{DATASET_DIR}/sd_xl_base_1.0.safetensors"

    if not os.path.exists(model_path):
        print("⚠️ Модель не найдена в датасете, используем HuggingFace...")
        model_path = "stabilityai/stable-diffusion-xl-base-1.0"

    print("Загрузка модели SDXL...")
    pipe = StableDiffusionXLPipeline.from_single_file(
        model_path,
        torch_dtype=torch.float16,
        use_safetensors=True
    ).to("cuda")

    pipe.enable_attention_slicing()
    pipe.enable_vae_slicing()

    print("✓ Модель загружена!\n")

    # Генерируем кадры
    start_time = time.time()
    for i in range(NUM_FRAMES):
        print(f"Кадр {i+1}/{NUM_FRAMES}...", end=" ")

        generator = torch.Generator(device="cuda").manual_seed(42 + i)

        image = pipe(
            prompt=PROMPT,
            negative_prompt=NEGATIVE_PROMPT,
            width=WIDTH,
            height=HEIGHT,
            num_inference_steps=STEPS,
            guidance_scale=CFG_SCALE,
            generator=generator
        ).images[0]

        image.save(f"{FRAMES_DIR}/{i}.png")
        print(f"✓ ({time.time() - start_time:.1f}s)")

    total_gen_time = time.time() - start_time
    print(f"\n✅ {NUM_FRAMES} кадров за {total_gen_time:.1f}s!\n")

    del pipe
    torch.cuda.empty_cache()

print(f"✓ Кадры сохранены в: {FRAMES_DIR}")

In [ ]:
# === UPSCALE С REAL-ESRGAN (ОПЦИОНАЛЬНО) ===
print_separator()
print("📈 АПСКЕЙЛ КАДРОВ С REAL-ESRGAN")
print_separator(nl_before=False)

# === CELL 1: minimal guards (idempotent when run separately) ===
import os, sys, shutil, subprocess
if 'WORKSPACE' not in globals():
    WORKSPACE = os.getcwd()
    print(f"⚠️ WORKSPACE not set — using {WORKSPACE}")
if 'FRAMES_DIR' not in globals():
    FRAMES_DIR = f"{WORKSPACE}/frames"
    print(f"⚠️ FRAMES_DIR not set — using {FRAMES_DIR}")
if 'DATASET_DIR' not in globals():
    DATASET_DIR = f"{WORKSPACE}/dataset"
    print(f"⚠️ DATASET_DIR not set — using {DATASET_DIR}")

os.makedirs(FRAMES_DIR, exist_ok=True)
os.makedirs(DATASET_DIR, exist_ok=True)

UPSCALE_MODEL_NAME = globals().get('UPSCALE_MODEL_NAME', '4x-UltraSharp.pth')
upscale_model = f"{DATASET_DIR}/{UPSCALE_MODEL_NAME}"
print('upscale_model =', upscale_model)

# === CELL 2: repo check & optional clone ===
REPO_DIR = os.path.join(WORKSPACE, 'Real-ESRGAN')
print('REPO_DIR =', REPO_DIR)
if os.path.exists(upscale_model):
    print('Upscale model found at', upscale_model)
    if not os.path.exists(REPO_DIR):
        print('Cloning Real-ESRGAN into', REPO_DIR)
        try:
            subprocess.check_call(['git', 'clone', 'https://github.com/xinntao/Real-ESRGAN', REPO_DIR])
            print('Clone OK')
        except Exception as e:
            print('⚠️ Clone failed:', e)
else:
    print('Upscale model not present — to run upscale place model at', upscale_model)

# === CELL 3: chdir to repo and prepare logs/install_cmd ===
if os.path.exists(REPO_DIR):
    os.chdir(REPO_DIR)
    print('CWD ->', os.getcwd())
else:
    print('Repo not present; skipping chdir')

LOG_DIR = os.path.join(WORKSPACE, 'logs')
os.makedirs(LOG_DIR, exist_ok=True)
INSTALL_LOG = os.path.join(LOG_DIR, 'real_esrgan_install.log')
INFERENCE_LOG = os.path.join(LOG_DIR, 'real_esrgan_inference.log')
install_cmd = [sys.executable, '-m', 'pip', 'install', '--upgrade', 'torchvision', '-f', 'https://download.pytorch.org/whl/torch_stable.html']
print('INFERENCE_LOG =', INFERENCE_LOG)

# === CELL 4a: build src_candidates list ===
src_candidates = []
if 'upscale_model' in globals() and upscale_model:
    src_candidates.append(upscale_model)
src_candidates.append(os.path.join(DATASET_DIR, UPSCALE_MODEL_NAME))
src_candidates.append(os.path.join(WORKSPACE, 'dataset', UPSCALE_MODEL_NAME))
src_candidates.append(os.path.join(WORKSPACE, 'models', UPSCALE_MODEL_NAME))
print('src_candidates:')
for p in src_candidates:
    print(' -', p)

# === CELL 4b: copy first existing candidate into weights/ ===
if os.path.exists(REPO_DIR) and src_candidates:
    try:
        weights_dir = os.path.join(os.getcwd(), 'weights')
        os.makedirs(weights_dir, exist_ok=True)
        copied = False
        for src in src_candidates:
            try:
                if src and os.path.exists(src):
                    shutil.copy(src, weights_dir)
                    print('Copied', src, '->', weights_dir)
                    copied = True
                    break
            except Exception as _e:
                print('Warning copying', src, ':', _e)
        if not copied:
            print('No candidate found to copy into weights')
    except Exception as e:
        print('⚠️ Error preparing weights:', e)
else:
    print('Skipping weights copy (repo or candidates missing)')

# === CELL 5a: candidate_shims list ===
candidate_shims = [
    os.path.join(WORKSPACE, 'tmp_inference_shim.py'),
    os.path.join(os.getcwd(), 'tmp_inference_shim.py'),
    os.path.join('/kaggle/working', 'tmp_inference_shim.py'),
    os.path.join('/apps/ComfyCloud_My_Work_Flow', 'tmp_inference_shim.py'),
]
print('candidate_shims:')
for c in candidate_shims:
    print(' -', c)

# === CELL 5b: try copy prepared shim ===
shim_path = os.path.join(os.getcwd(), 'inference_with_shim_full.py')
wrote = False
for c in candidate_shims:
    try:
        if c and os.path.exists(c):
            shutil.copy(c, shim_path)
            wrote = os.path.exists(shim_path)
            print('Copied shim from', c)
            break
    except Exception as e:
        print('Failed to copy shim from', c, ':', e)
print('shim_path exists:', os.path.exists(shim_path))

# === CELL 6a: build shim content (parts) ===
shim_lines_part1 = 'import sys, os, runpy, types, re'
shim_lines_part2 = '# Minimal compatibility shim: define netscale/model/outscale and exec the original script'
shim_lines_part3 = 'def _infer_netscale(argv):'
shim_lines_part4 = '    for i,a in enumerate(argv):'
print('shim parts defined')

# === CELL 6b: compose shim_lines and write shim file (if needed) ===
if (not os.path.exists(shim_path)) and os.path.exists(REPO_DIR):
    shim_lines = [
        shim_lines_part1,
        shim_lines_part2,
        shim_lines_part3,
        shim_lines_part4,
        "    if a in ('-n','--name','--model') and i+1<len(argv):",
        "        m = re.match(r'(\\d+)x', argv[i+1])",
        "        if m:",
        "            try: return int(m.group(1))\\n            except: pass",
        "    for i,a in enumerate(argv):",
        "        if a in ('-s','--scale') and i+1<len(argv):",
        "            try: return int(argv[i+1])\\n            except: pass",
        "    return 4",
        "if __name__ == '__main__':",
        "    argv = sys.argv[1:]",
        "    netscale = _infer_netscale(argv)",
        "    script_path = os.path.join(os.getcwd(), 'inference_realesrgan.py')",
        "    try:",
        "        with open(script_path, 'r', encoding='utf-8') as f:",
        "            code = f.read()",
        "    except Exception:",
        "        sys.argv = [script_path] + argv",
        "        runpy.run_path('inference_realesrgan.py', run_name='__main__')",
        "    else:",
        "        _globals = {'__name__':'__main__', '__file__':script_path, 'netscale':netscale, 'model':None, 'outscale':netscale, 'scale':netscale}",
        "        _globals['sys'] = sys",
        "        sys.argv = [script_path] + argv",
        "        exec(compile(code, script_path, 'exec'), _globals)",
    ]
    try:
        with open(shim_path, 'w', encoding='utf-8') as sf:
            sf.write('\n'.join(shim_lines))
        wrote = os.path.exists(shim_path)
        if wrote:
            print('Wrote shim to', shim_path)
    except Exception as e:
        print('Warning: failed to write shim:', e)
else:
    print('Shim exists or repo missing; shim_path exists =', os.path.exists(shim_path))

# === CELL 7a: construct fallback safe_argv & args_literal ===
safe_argv = ['-n','4x-UltraSharp','-i', FRAMES_DIR, '-o', FRAMES_DIR + '_upscaled', '--fp32', '--outscale', '4']
args_literal = '[' + ','.join(repr(a) for a in safe_argv) + ']'
print('safe_argv ->', safe_argv)

# === CELL 7b: build run_cmd (shim vs fallback) — inspect but don't run ===
run_cmd = None
if os.path.exists(REPO_DIR):
    shim_path = os.path.join(os.getcwd(), 'inference_with_shim_full.py')
    if os.path.exists(shim_path):
        run_cmd = [sys.executable, shim_path, '-n','4x-UltraSharp','-i',FRAMES_DIR,'-o',FRAMES_DIR + '_upscaled','--fp32','--outscale','4']
    else:
        pycmd = (
            "import runpy,sys,os,re; argv=" + args_literal + "; sys.argv=[os.path.join(os.getcwd(),'inference_realesrgan.py')]+argv; "
            "def _infer(argv):\n    import re\n    for i,a in enumerate(argv):\n        if a in ('-n','--name','--model') and i+1<len(argv):\n            m=re.match(r'(\\d+)x',argv[i+1]);\n            if m: return int(m.group(1))\n    for i,a in enumerate(argv):\n        if a in ('-s','--scale') and i+1<len(argv):\n            try: return int(argv[i+1])\n            except: pass\n    return 4\n"
            "netscale=_infer(argv)\nrunpy.run_path('inference_realesrgan.py', run_name='__main__')"
        )
        run_cmd = [sys.executable, '-c', pycmd]
print('Prepared run_cmd:')
print(run_cmd)
print('\nWhen ready — run the next cell to execute inference.')

# === CELL 8: execute run_cmd (inference) — run only after inspection ===
if run_cmd is None:
    print('run_cmd undefined — build it first (previous cell)')
else:
    print('Running:', run_cmd)
    try:
        res = subprocess.run(run_cmd, capture_output=True, text=True, timeout=3600)
    except Exception as e:
        print('Failed to run subprocess:', e)
        res = None
    if res is not None:
        try:
            with open(INFERENCE_LOG, 'a', encoding='utf-8') as lf:
                lf.write('=== STDOUT ===\n')
                lf.write(res.stdout or '')
                lf.write('\n=== STDERR ===\n')
                lf.write(res.stderr or '')
        except Exception:
            pass
        print('Return code =', getattr(res,'returncode',None))
        if res.returncode == 0:
            print('✓ Inference completed — see', INFERENCE_LOG)
        else:
            print('✗ Inference failed — see', INFERENCE_LOG)
            globals()['last_inference_stderr'] = res.stderr

# === CELL 9a: If stderr mentions functional_tensor — prepare minimal stub strings ===
# (define strings separately so user can inspect before writing)
if 'last_inference_stderr' in globals():
    _stderr_preview = (globals().get('last_inference_stderr') or '').lower()
else:
    _stderr_preview = ''
print('stderr preview contains functional_tensor:', 'functional_tensor' in _stderr_preview)

# Prepare stub strings (do not write yet)
tv_init = """# Minimal stub torchvision package for Real-ESRGAN/basicsr compatibility
from . import utils
from . import transforms
__all__ = ['utils','transforms']
"""

utils_code = """# Minimal stub for torchvision.utils.make_grid
try:
    from torchvision.utils import make_grid as _make_grid
    def make_grid(tensor, nrow=8, padding=2, normalize=False):
        return _make_grid(tensor, nrow=nrow, padding=padding, normalize=normalize)
except Exception:
    import numpy as _np
    from PIL import Image as _Image
    import torch as _torch
    def make_grid(tensor, nrow=8, padding=2, normalize=False):
        if isinstance(tensor, (list, tuple)):
            tensor = _torch.stack(tensor, dim=0)
        if not isinstance(tensor, _torch.Tensor):
            raise TypeError('Fallback make_grid expects a torch.Tensor or list of tensors')
        t = tensor.detach().cpu()
        if t.dim() == 3:
            t = t.unsqueeze(0)
        B,C,H,W = t.shape
        if normalize:
            t = (t - t.min()) / (t.max() - t.min() + 1e-8)
        else:
            if t.max() > 50:
                t = t / 255.0
        def to_uint8(x):
            arr = (x.numpy().transpose(1,2,0) * 255.0).clip(0,255).astype(_np.uint8)
            if arr.shape[2] == 1:
                arr = _np.repeat(arr, 3, axis=2)
            return _Image.fromarray(arr)
        imgs = [to_uint8(t[i]) for i in range(B)]
        rows = (B + nrow - 1) // nrow
        grid_h = rows * H + padding * (rows - 1)
        grid_w = nrow * W + padding * (nrow - 1)
        grid = _Image.new('RGB', (grid_w, grid_h), (0,0,0))
        for idx, img in enumerate(imgs):
            r = idx // nrow
            c = idx % nrow
            grid.paste(img, (c * (W + padding), r * (H + padding)))
        arr = _np.array(grid).transpose(2,0,1).astype(_np.float32) / 255.0
        return _torch.from_numpy(arr)
"""

tr_init = """# transforms package stub
from . import functional_tensor
__all__ = ['functional_tensor']
"""

func_code = """# Auto-generated stub: try to import the real functional_tensor, otherwise delegate to torchvision.transforms.functional
try:
    from torchvision.transforms.functional_tensor import *  # type: ignore
except Exception:
    try:
        from torchvision.transforms import functional as _f
    except Exception:
        def rgb_to_grayscale(x):
            raise ImportError('rgb_to_grayscale not available in this environment')
        def convert_image_dtype(x, dtype):
            raise ImportError('convert_image_dtype not available in this environment')
    else:
        rgb_to_grayscale = getattr(_f, 'rgb_to_grayscale', None)
        convert_image_dtype = getattr(_f, 'convert_image_dtype', None)
    __all__ = [n for n in ('rgb_to_grayscale','convert_image_dtype') if globals().get(n) is not None]
"""

print('Prepared stub strings (not written yet)')

# === CELL 9b: write stub files and retry run_cmd with PYTHONPATH adjusted ===
if 'functional_tensor' in _stderr_preview or 'functional_tensor' in (tail(INFERENCE_LOG, 50) or '').lower():
    try:
        stub_root = os.path.join(WORKSPACE, 'torchvision_stub')
        torchvision_pkg = os.path.join(stub_root, 'torchvision')
        transforms_pkg = os.path.join(torchvision_pkg, 'transforms')
        shutil.rmtree(stub_root, ignore_errors=True)
        os.makedirs(transforms_pkg, exist_ok=True)

        with open(os.path.join(torchvision_pkg, '__init__.py'), 'w', encoding='utf-8') as f_init:
            f_init.write(tv_init)
        with open(os.path.join(torchvision_pkg, 'utils.py'), 'w', encoding='utf-8') as f_utils:
            f_utils.write(utils_code)
        with open(os.path.join(transforms_pkg, '__init__.py'), 'w', encoding='utf-8') as f_tr:
            f_tr.write(tr_init)
        with open(os.path.join(transforms_pkg, 'functional_tensor.py'), 'w', encoding='utf-8') as f_ft:
            f_ft.write(func_code)

        env = os.environ.copy()
        prev_pp = env.get('PYTHONPATH', '')
        env['PYTHONPATH'] = stub_root + (os.pathsep + prev_pp if prev_pp else '')

        print('Running run_cmd with stub PYTHONPATH...')
        if run_cmd is None:
            print('run_cmd undefined — cannot retry')
        else:
            res_stub = subprocess.run(run_cmd, capture_output=True, text=True, timeout=3600, env=env)
            print('Stub attempt returncode:', getattr(res_stub,'returncode',None))
            try:
                with open(INFERENCE_LOG, 'a', encoding='utf-8') as lf:
                    lf.write('\n=== STUB SUBPROCESS STDOUT ===\n')
                    lf.write(res_stub.stdout or '')
                    lf.write('\n=== STUB SUBPROCESS STDERR ===\n')
                    lf.write(res_stub.stderr or '')
            except Exception:
                pass
            if res_stub.returncode == 0:
                print('✓ Inference completed using local stub; see', INFERENCE_LOG)
            else:
                print('Stub attempt failed with code', res_stub.returncode)
                print('Attempting editable install of Real-ESRGAN package then torchvision install and retry...')
                try:
                    with open(INFERENCE_LOG, 'a', encoding='utf-8') as lf:
                        lf.write('\n=== Ensuring Real-ESRGAN package is installed (editable) ===\n')
                except Exception:
                    pass
                try:
                    res_pkg = subprocess.run([sys.executable, '-m', 'pip', 'install', '-e', '.'], capture_output=True, text=True, timeout=900)
                    with open(INFERENCE_LOG, 'a', encoding='utf-8') as lf:
                        lf.write('\n=== PKG INSTALL STDOUT ===\n')
                        lf.write(res_pkg.stdout or '')
                        lf.write('\n=== PKG INSTALL STDERR ===\n')
                        lf.write(res_pkg.stderr or '')
                except Exception as _pkg_e:
                    print('Editable install failed:', _pkg_e)

                try:
                    res_install = subprocess.run(install_cmd, capture_output=True, text=True, timeout=900)
                    with open(INFERENCE_LOG, 'a', encoding='utf-8') as lf:
                        lf.write('\n=== TORCHVISION INSTALL STDOUT ===\n')
                        lf.write(res_install.stdout or '')
                        lf.write('\n=== TORCHVISION INSTALL STDERR ===\n')
                        lf.write(res_install.stderr or '')
                except Exception as _ie:
                    print('torchvision install failed:', _ie)
                    res_install = None

                if res_install and res_install.returncode == 0:
                    print('torchvision installed; retrying inference (with stub PYTHONPATH)')
                    res2 = subprocess.run(run_cmd, capture_output=True, text=True, timeout=3600, env=env)
                    with open(INFERENCE_LOG, 'a', encoding='utf-8') as lf:
                        lf.write('\n=== RETRY STDOUT ===\n')
                        lf.write(res2.stdout or '')
                        lf.write('\n=== RETRY STDERR ===\n')
                        lf.write(res2.stderr or '')
                    if res2.returncode == 0:
                        print('✓ Real-ESRGAN inference completed after installing torchvision; see', INFERENCE_LOG)
                    else:
                        print('Retry also failed — see', INFERENCE_LOG)
                else:
                    print('Could not install torchvision — see', INFERENCE_LOG)
    except Exception as e:
        print('Exception during stub/install/retry:', e)
else:
    print('No functional_tensor error detected in stderr tail — skip stub flow')

# === CELL 10: final move/rename upscaled frames into FRAMES_DIR (if present) ===
try:
    if os.path.exists(FRAMES_DIR + '_upscaled'):
        print('Moving upscaled frames into', FRAMES_DIR)
        shutil.rmtree(FRAMES_DIR, ignore_errors=True)
        shutil.move(FRAMES_DIR + '_upscaled', FRAMES_DIR)
        import glob
        upscaled_files = sorted(glob.glob(f"{FRAMES_DIR}/*_out.png"))
        for i, filepath in enumerate(upscaled_files):
            try:
                os.rename(filepath, f"{FRAMES_DIR}/{i}.png")
            except Exception:
                pass
        print('✓ Upscaled frames moved/renamed')
    else:
        print('No upscaled frames dir found — nothing to move')
except Exception as e:
    print('Error while moving/renaming upscaled frames:', e)


In [ ]:
# === ИТОГИ ===
# Защитная проверка: если переменная total_gen_time не определена (ячейка генерации не запускалась),
# ставим безопасное значение 0.0
if 'total_gen_time' not in globals():
    total_gen_time = 0.0

print_separator()
print("📋 ИТОГИ ГЕНЕРАЦИИ")
print_separator(nl_before=False)
print(f"Режим: {'AnimateDiff (анимация)' if USE_ANIMATEDIFF else 'Статичные кадры'}")
print(f"Промпт: {PROMPT[:50]}...")
print(f"Размер: {WIDTH}x{HEIGHT}")
print(f"Кадров: {NUM_FRAMES}" + (f" → {NUM_FRAMES * (2**RIFE_EXP)}" if USE_RIFE else ""))
print(f"Время генерации: {total_gen_time:.1f}s")

if USE_ANIMATEDIFF:
    print(f"\n{'="*60}")
    print("💡 СОВЕТЫ ДЛЯ УЛУЧШЕНИЯ АНИМАЦИИ")
    print(f"{'='*60}")
    print("Добавьте в промпт:")
    print("  • 'turning head', 'blinking', 'hair flowing'")
    print("  • 'smooth motion', 'cinematic camera movement'")
    print("  • 'dynamic pose', 'wind blowing'")
    print("\nДобавьте в negative:")
    print("  • 'static', 'frozen', 'choppy animation'")
    print("  • 'stiff', 'rigid', 'still image'")

print(f"\n{'='*60}")
print("✅ ВСЁ ГОТОВО!")
print(f"{'='*60}")


In [ ]:
# === UNLOAD PIPE (освобождение pipeline и VRAM) ===
# Нажмите кнопку, чтобы удалить `pipe` и `adapter` из globals и очистить VRAM.
def unload_pipe():
    """Удаляет pipe/adapter из глобальной области и очищает VRAM (если доступен torch)."""
    import gc
    removed = []
    try:
        if 'pipe' in globals():
            try:
                del globals()['pipe']
                removed.append('pipe')
            except Exception:
                pass
        if 'adapter' in globals():
            try:
                del globals()['adapter']
                removed.append('adapter')
            except Exception:
                pass
        # Попробуем освободить GPU память
        try:
            import torch
            torch.cuda.empty_cache()
            removed.append('cuda_cache_cleared')
        except Exception:
            pass
        # Общая уборка памяти
        gc.collect()
        print(f"✓ Удалено: {', '.join(removed) if removed else 'ничего не найдено'}")
    except Exception as e:
        print('⚠️ Ошибка при выгрузке pipe:', e)

# Показываем кнопку если ipywidgets доступен, иначе показываем инструкцию
try:
    import ipywidgets as widgets
    from IPython.display import display
    btn_unload = widgets.Button(description='Unload pipe (free VRAM)', button_style='warning')
    def _on_unload_click(b):
        unload_pipe()
    btn_unload.on_click(_on_unload_click)
    display(btn_unload)
except Exception:
    print('\nℹ️ Для быстрого освобождения памяти выполните в ячейке: unload_pipe()')


In [ ]:
# === QUICK SMOKE GENERATION — быстрый placeholder для проверки workflow ===
# Создаёт простые кадры с движущимся текстом/элементом и собирает MP4 через ffmpeg.
import os
import subprocess
from PIL import Image, ImageDraw, ImageFont

print_separator()
print('⚡ QUICK SMOKE GENERATION — создаём placeholder-кадры')
print_separator(nl_before=False)

# Параметры (безопасные небольшие значения)
SMOKE_WIDTH = 256
SMOKE_HEIGHT = 256
SMOKE_FRAMES = 8
SMOKE_FPS = 8

# Путь для кадров
if 'WORKSPACE' not in globals():
    WORKSPACE = os.getcwd()
SMOKE_FRAMES_DIR = f"{WORKSPACE}/smoke_frames"
os.makedirs(SMOKE_FRAMES_DIR, exist_ok=True)

# Промпт для отображения
prompt_text = globals().get('PROMPT', 'Test generation — change PROMPT cell')
text_preview = (prompt_text or '')[:120]

# Генерируем кадры
for i in range(SMOKE_FRAMES):
    # фон и градиент
    r = (30 + i * 20) % 256
    g = (80 + i * 10) % 256
    b = (160 + i * 5) % 256
    img = Image.new('RGB', (SMOKE_WIDTH, SMOKE_HEIGHT), (r, g, b))
    draw = ImageDraw.Draw(img)
    try:
        font = ImageFont.load_default()
    except Exception:
        font = None
    # движущийся текст
    w, h = draw.textsize(text_preview, font=font)
    x = int((SMOKE_WIDTH - w) * i / max(1, SMOKE_FRAMES - 1))
    y = SMOKE_HEIGHT // 2 - h // 2
    draw.text((x, y), text_preview, fill=(255, 255, 255), font=font)
    # движущийся кружок
    cx = int(SMOKE_WIDTH * (0.2 + 0.6 * (i / max(1, SMOKE_FRAMES - 1))))
    cy = int(SMOKE_HEIGHT * 0.25)
    r0 = 12
    draw.ellipse((cx - r0, cy - r0, cx + r0, cy + r0), fill=(255, 200, 0))
    # подпись кадра
    draw.text((6, SMOKE_HEIGHT - 14), f'frame {i+1}/{SMOKE_FRAMES}', fill=(230,230,230), font=font)
    path = os.path.join(SMOKE_FRAMES_DIR, f"{i}.png")
    img.save(path)
    print(f'  saved {path}')

# Собираем MP4 через ffmpeg
smoke_video = f"{WORKSPACE}/SMOKE_OUTPUT.mp4"
ffmpeg_cmd = [
    'ffmpeg', '-y', '-framerate', str(SMOKE_FPS), '-i', f"{SMOKE_FRAMES_DIR}/%d.png",
    '-c:v', 'libx264', '-pix_fmt', 'yuv420p', '-preset', 'fast', smoke_video
]
print('\nЗапуск ffmpeg...')
try:
    subprocess.check_call(ffmpeg_cmd, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    print(f'✓ Smoke video saved: {smoke_video}')
    try:
        from IPython.display import FileLink, display
        display(FileLink(smoke_video))
    except Exception:
        pass
except Exception as e:
    print('⚠️ ffmpeg failed or not available:', e)
    print('  You can manually assemble frames using:')
    print(f"  ffmpeg -framerate {SMOKE_FPS} -i {SMOKE_FRAMES_DIR}/%d.png -c:v libx264 -pix_fmt yuv420p -preset fast {smoke_video}")

print_separator()


In [ ]:
# === SHOW REAL-ESRGAN LOGS ===
# Показывает начало логов установки и инференса Real-ESRGAN, если они существуют
import os
LOG_DIR = os.path.join(os.getcwd(), 'logs')
install_log = os.path.join(LOG_DIR, 'real_esrgan_install.log')
inf_log = os.path.join(LOG_DIR, 'real_esrgan_inference.log')

def show_log(path, max_chars=20000):
    if os.path.exists(path):
        print('\n' + '='*40)
        print('LOG:', path)
        print('='*40)
        try:
            with open(path, 'r', encoding='utf-8', errors='replace') as f:
                data = f.read()
            print(data[:max_chars])
            if len(data) > max_chars:
                print('\n... (truncated) ...')
        except Exception as e:
            print('Could not read log:', e)
    else:
        print(f'Log not found: {path}')

print('Checking Real-ESRGAN logs in', LOG_DIR)
show_log(install_log)
show_log(inf_log)
